In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

g:\Mi unidad\SONY-GATEWAY\MaLB-SC-generation-module\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import dspy

g:\Mi unidad\SONY-GATEWAY\MaLB-SC-generation-module\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
gpt4_turbo = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=300)
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300, temperature=1)

dspy.settings.configure(lm=gpt3_turbo, max_tokens=1024)

In [67]:
A1 = """
When a buyer requests a refund, the smart contract should compensate them fairly. To achieve this, the contract needs to consider the buyer and the current state of the sale, such as whether it's in progress or cancelled. Only the seller is authorized to initiate the refund process, ensuring that refunds are legitimate and secure. In the primary scenario, if the sale is in progress, the buyer should receive a refund of twice the value they paid initially if the seller decides not to proceed with the sale. However, if the sale is not in progress, no refund will be issued. This requirement ensures that buyers are fairly compensated in case of a sale cancellation, while also protecting sellers from unauthorized refunds
"""

A2 = """
A smart contract is a self-executing contract with the terms of the agreement between buyer and seller being directly written into lines of code. The code and the agreements contained therein exist across a distributed, decentralized blockchain network. The code controls the execution, and transactions are trackable and irreversible.
"""

A3 = """
I have a blockchain 3 business and I want to improve the efficiency of it by implementing smart contracts.
"""

In [54]:
import pydantic
description = A1

class IsAboutSmartContracts(pydantic.BaseModel):
    parameter: bool

class AssessSubject(dspy.Signature):
    """Verify that the description talks about Smart Contracts"""
    smart_contract_description: str = dspy.InputField(desc="A description of a Smart Contract")
    boolean_assessment: IsAboutSmartContracts = dspy.OutputField(desc="True/False indicating if text is about Smart Contracts")

class AssessSubjectModule(dspy.Module):
    """A module to verify if the description is about Smart Contracts"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedChainOfThought(AssessSubject)

    def forward(self, description: str) -> IsAboutSmartContracts:
        pred = self.generate_answer(smart_contract_description=description)
        return pred.boolean_assessment.parameter

# Example usage
module = AssessSubjectModule()
result = module.forward(description)
print(result)

# gpt3_turbo.inspect_history(n=1)

True


In [70]:
import pydantic
description = A1

# Pydantic model to define the output structure
class OutputRate(pydantic.BaseModel):
    parameter: int

class RateCompexity(dspy.Signature):
    """Rate the technical completeness of the smart contract description on a scale of 1 to 10"""
    smart_contract_description: str = dspy.InputField(desc="A description of a Smart Contract")
    complexity_score: OutputRate = dspy.OutputField(desc="A rating from 1 to 10")

class RateComplexityModule(dspy.Module):
    """A module to rate the technical completeness of a smart contract description"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedChainOfThought(RateCompexity)

    def forward(self, description: str) -> int:
        pred = self.generate_answer(smart_contract_description=description)
        return pred.complexity_score.parameter

# Example usage of the module
module = RateComplexityModule()
rating = module.forward(description)
print(rating)

# gpt3_turbo.inspect_history(n=1)


### COMPILATION ########################################################################

# must match "forward" function arguments
trainset = [
    dspy.Example(smart_contract_description=description, complexity_score=0).with_inputs("description"),
]

# first_data_point = trainset[0]
# print(first_data_point.inputs())
# print(first_data_point.labels())

from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
def validate_assessment(example, pred, trace=None):
    ### answer_EM = dspy.evaluate.answer_exact_match(smart_contract_description, complexity_score)
    answer_match = example.complexity_score == pred.complexity_score
    return answer_match

teleprompter = BootstrapFewShot(metric=validate_assessment) # or dspy.evaluate.answer_exact_amtch
compiled_module = teleprompter.compile(RateComplexityModule(), trainset=trainset)
# compiled_module.save(".")
# loaded_program = YOUR_PROGRAM_CLASS()
# loaded_program.load(path=YOUR_SAVE_PATH)

devset = [
    dspy.Example(smart_contract_description=description, complexity_score=5).with_inputs("description")
]

print('#')

from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
evaluator(compiled_module, metric=validate_assessment)

8


AttributeError: module 'dspy' has no attribute 'BootstrapFewShot'

In [57]:
import pydantic
from typing import List
description = A1

class RequirementsList(pydantic.BaseModel):
    parameter: List[str]

class ExtractRequirements(dspy.Signature):
    """Extract requirements from a Smart Contract Description"""
    smart_contract_description: str = dspy.InputField(desc="A description of a Smart Contract")
    requirements_list: RequirementsList = dspy.OutputField(desc="A list object with extracted requirements")

class ExtractRequirementsModule(dspy.Module):
    """A module to extract requirements from a Smart Contract Description"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(ExtractRequirements)

    def forward(self, description: str) -> RequirementsList:
        pred = self.generate_answer(smart_contract_description=description)
        return pred.requirements_list.parameter

module = ExtractRequirementsModule()
list_of_requirements = module.forward(description)
print(list_of_requirements)

# gpt3_turbo.inspect_history(n=1)

['buyer request refund', 'compensation based on current state of sale (in progress or cancelled)', 'seller authorization for refund initiation', 'refund amount calculation based on sale state', 'no refund if sale is not in progress']


In [42]:
import pydantic
from typing import List

class Requirement(pydantic.BaseModel):
    Name: str
    Scope: str
    Input: List[str]
    Constraints: str
    Output: str
    PrimaryScenario: str
    AlternativeScenario: str

class generate_attributes(dspy.Signature):
    """Generate attributes for the given attribute description."""
    smart_contract_description: str = dspy.InputField(desc="Context of the requirement")
    requirement_description: str = dspy.InputField()
    structured_requirement: Requirement = dspy.OutputField(desc="Structured list of requirement attributes")

class GenerateAttributesModule(dspy.Module):
    """A module to process multiple requirement descriptions into structured JSON."""

    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(generate_attributes)
    
    def forward(self, description: str, requirement: str) -> Requirement:
        pred = self.generate_answer(
            smart_contract_description=description,
            requirement_description=requirement
            )
        return pred.structured_requirement

req_module = GenerateAttributesModule()
# List of requirements has multiple strings inside.
for requirement in list_of_requirements:
    attributes = req_module.forward(description, requirement)
    print(attributes.dict())

# gpt3_turbo.inspect_history(n=1)

{'Name': 'Buyer Refund Request', 'Scope': 'Smart contract for refund process', 'Input': ['Buyer request', 'Sale status'], 'Constraints': 'Only seller can initiate refund process', 'Output': 'Fair compensation for buyer', 'PrimaryScenario': 'If sale in progress, refund of twice the initial value', 'AlternativeScenario': 'No refund if sale not in progress'}
{'Name': 'Compensation based on current state of sale', 'Scope': 'Smart contract refund process', 'Input': ['Buyer request for refund', 'Current state of the sale (in progress or cancelled)'], 'Constraints': 'Only seller authorized to initiate refund process', 'Output': 'Fair compensation for buyer if sale is cancelled', 'PrimaryScenario': 'If sale is in progress and seller decides not to proceed, buyer receives a refund of twice the value paid initially', 'AlternativeScenario': 'No refund issued if sale is not in progress'}
{'Name': 'Seller Authorization for Refund Initiation', 'Scope': 'Smart Contract for Buyer Refunds', 'Input': ['